In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Read data
df_1m = pd.read_parquet("../data/silver/power_load_1m.parquet")

In [3]:
def plot_load_data(df, title_suffix):
    # -----------------------------
    # Ensure types
    # -----------------------------
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="raise")
    df = df.sort_values("timestamp")

    # -----------------------------
    # Create day label + seconds-of-day
    # -----------------------------
    df["date"] = df["timestamp"].dt.normalize()  # midnight of that date
    df["second_of_day"] = (df["timestamp"] - df["date"]).dt.total_seconds().astype(int)

    # -----------------------------
    # Select which days to plot (first 6 by date)
    # -----------------------------
    unique_dates = df["date"].drop_duplicates().sort_values()
    selected_dates = unique_dates.iloc[:6]   # like MATLAB [1:6]
    resolution = 1

    fig = go.Figure()

    for d in selected_dates:
        day_df = df[df["date"] == d].sort_values("second_of_day")
        day_df = day_df.iloc[::resolution]

        fig.add_trace(
            go.Scatter(
                x=day_df["second_of_day"].to_numpy() / 3600,
                y=day_df["avg_load"].to_numpy() / 1e3,
                mode="lines",
                name=str(d.date()),
            )
        )

    fig.update_layout(
        title=f"Daily power/demand data over {len(selected_dates)} day(s) - {title_suffix}",
        xaxis_title=f"time [hours in {resolution} sec. resolution]",
        yaxis_title="power/demand [kW]",
        template="plotly_white",
    )

    fig.show()

plot_load_data(df_1m, "1-minute resolution")

In [5]:
# average load per day of week
df_1m["day_of_week"] = df_1m["timestamp"].dt.day_name()
avg_load_by_day = df_1m.groupby("day_of_week")["avg_load"].mean().reindex([
    "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
])
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=avg_load_by_day.index,
        y=avg_load_by_day.values / 1e3,
        name="Average Load"
    )
)
fig.update_layout(
    title="Average Power/Demand by Day of Week",
    xaxis_title="Day of Week",
    yaxis_title="Average Power/Demand [kW]",
    template="plotly_white",
)

In [18]:
# Average Load Per Workday (workday field) (0 = Not workday, 1 = Full Workday, 2 = Half workday)
avg_load_by_workday = df_1m.groupby("workday")["avg_load"].mean()
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=["Not Workday", "Half Workday", "Full Workday"],
        y=avg_load_by_workday.values / 1e3,
        name="Average Load"
    )
)
fig.update_layout(
    title="Average Power/Demand by Workday Type",
    xaxis_title="Workday Type",
    yaxis_title="Average Power/Demand [kW]",
    template="plotly_white",
)

In [17]:
import plotly.graph_objects as go

# Compute average hourly profile per workday type
profiles = (
    df_1m
    .assign(hour=lambda x: x["timestamp"].dt.hour)
    .groupby(["workday", "hour"])["avg_load"]
    .mean()
    .reset_index()
)

fig = go.Figure()

workday_labels = {
    0: "Non-Workday",
    1: "Half Workday",
    2: "Full Workday"
}

colors = {
    0: "orange",
    1: "steelblue",
    2: "seagreen"
}

for wd in [0, 1, 2]:
    df_wd = profiles[profiles["workday"] == wd]
    fig.add_trace(
        go.Bar(
            x=df_wd["hour"],
            y=df_wd["avg_load"] / 1e3,
            name=workday_labels[wd],
            marker_color=colors[wd],
        )
    )

fig.update_layout(
    title="Average Daily Load Profile by Workday Type (1-Minute Resolution)",
    xaxis_title="Hour of Day",
    yaxis_title="Average Power/Demand [kW]",
    barmode="group",
    template="plotly_white",
)

fig.show()
